# Requests - Seeking alpha API

## Rapid API

We will use Rapidapi.com's API to request Seeking Alpha (seekingalpha.com) content. Seeking Alpha is a trading website (graphs, news, SEC filings, etc). An API (Application programming interface) is a light way (in terms of #bytes and number of requests needed) to send/receive data compared to getting the same data off a web page.

### Please register for a free account on https://rapidapi.com/apidojo/api/seeking-alpha/pricing

Then select the 'basic plan', which allows for 500 free API calls. (Note that this could go quick: when writing/debugging your code limit the number of API calls so that you don't run out.)

Click on 'endpoints' (first menu item on the top left); your API key is in the middle panel (where it says 'Header parameters' and 'X-RapidAPI-Key').
                                                               


### API Key

Rename file 'api-key-example.txt' to 'api-key.txt' and paste in your API key

See page https://rapidapi.com/apidojo/api/seeking-alpha 

### Read API key from disk

In the repository, I have used github settings to exclude (ignore) file 'api-key.txt'. This way, you can share your code publicly without exposing passwords.


In [ ]:
# read file (first line)
# this will only work with files that are one line
with open('api-key.txt', 'r') as file:
    apiKey = file.read().rstrip()

## Example request

In [ ]:
import requests
import json

In [ ]:
# symbols/get-meta-data is endpoint (see documentation of the various endpoints)
# documentation: https://rapidapi.com/apidojo/api/seeking-alpha
url = "https://seeking-alpha.p.rapidapi.com/symbols/get-meta-data"

headers = {
    'x-rapidapi-key': apiKey,
    'x-rapidapi-host': "seeking-alpha.p.rapidapi.com"
}

# in a 'get' requests the params are added to the url, in this case: ?symbol=aapl
params = {"symbol": "aapl" }

# request
r = requests.request("GET", url, headers=headers, params=params)

In [ ]:
# html response code, 200 is 'OK', 404 is not found, etc
r

In [ ]:
# status code - should be 200 ('OK')
r.status_code 

In [ ]:
# this is the server response in text format
r.text

In [ ]:
# this is the server response in json format
r.json()

In [ ]:
# let's wrap that in a few functions

# doRequest: 'low' level request
# url: string with url
# params: object (dictionary) that is sent with request
# apiKey: string with api-key
def doRequest( url, params, apiKey):
    # headers
    headers = {
    'x-rapidapi-key': apiKey,
    'x-rapidapi-host': "seeking-alpha.p.rapidapi.com"
    }
    
    return ( requests.get(url, headers=headers, params=params) )

def getMetaData (ticker, apiKey):
    params = {"symbol": ticker }
    return doRequest("https://seeking-alpha.p.rapidapi.com/symbols/get-meta-data", params, apiKey)
    

In [ ]:
r = getMetaData("aapl", apiKey)

In [ ]:
# should be 200 ('OK')
r.status_code 

In [ ]:
r.json()

## Get all transcript Ids

In [ ]:
# another endpoint (transcripts of conference calls)
# ticker: string
# apiKey: string
# number: number, default is 1 (if not passed in)
def getAllTranscriptIds( ticker, apiKey, number = 1):
    params = {"id": ticker, "size": 40, "number": number }
    return doRequest("https://seeking-alpha.p.rapidapi.com/transcripts/v2/list", params, apiKey)                      


In [ ]:
r = getAllTranscriptIds("aapl", apiKey)

In [ ]:
# status code - should be 200 ('OK')
r.status_code 

In [ ]:
# this is the server response in text format
r.text

### inspect in https://jsonviewer.stack.hu/

Notice the structure of the json:
    
```
{} JSON, means the whole structure is a dictionary (object)
[] data: on the JSON there is a data property which is a list, expand (click '+')
    {} 0, {} 1, etc, (data is a list of dictionaries), expand one
        id, type, {} attributes, {} relationships, {} link
[] included: there is a property included (list)
{} meta: there is a dictionary meta, when expanded (click on '+')
    {} page: there is a dictionary page, when expanded (click on '+')
        title: string
        description: string
        size: 40
        totalPages: 2
        total: 75
    {} minmaxPublishOn: dictionary
```

So there are 75 conference calls for Apple (aapl), and with 2 requests we would get these


In [ ]:
# page dictionary
# note that every endpoint can have a different structure; you need to inspect
# the structure before you can know the structure (don't assume any structure)
# (or use documentation)
page = r.json()["meta"]["page"]
page

In [ ]:
numPages = int(page["totalPages"])
print(numPages)

## Get a transcript

In [ ]:
# let's get the first transcript
id = r.json()["data"][0]["id"]
id

In [ ]:
# ticker: string
# id: string (or number) with transcript id
# ticker: string
# apiKey: string
# number: number, default is 1 (if not passed in)
def getTranscript( transcriptId, apiKey):
    params = {"id": transcriptId }
    return doRequest("https://seeking-alpha.p.rapidapi.com/transcripts/v2/get-details", params, apiKey)      

r = getTranscript(id, apiKey)


In [ ]:
print(r.text)

In [ ]:
# content
content = r.json()["data"]["attributes"]["content"]
print(content)